# feats using polar yay

In [67]:
import polars as pl
import yaml
from pathlib import Path

# pl.set_option("display", False)

In [68]:
input_path = Path("../data/cleaned/2015_2024_statcast_clean.parquet")

df = pl.read_parquet(input_path)

In [ ]:
df = df.sort(
    [
        "game_date",
        "game_pk",
        "at_bat_number",
        "pitch_number",
    ],
    descending=[False, False, False, True],
)


df.head()

df = df.with_columns(
    df.select(
        pl.col(pl.String).exclude(["player_name"]).cast(pl.Categorical).to_physical()
    ),
)

df
df = df.fill_null(-1)
df = df.sort(
    [
        "game_date",
        "game_pk",
        "at_bat_number",
        "pitch_number",
    ],
    descending=True,
)

df = df.with_columns(
    df.select(pl.col("pitch_type").shift(-1).alias("next_pitch")),
).drop_nulls("next_pitch")

df

pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,zone,stand,p_throws,type,hit_location,bb_type,balls,strikes,pfx_x,pfx_z,plate_x,plate_z,outs_when_up,inning,inning_topbot,hc_x,hc_y,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,if_fielding_alignment,of_fielding_alignment,spin_axis,bat_speed,swing_length,next_pitch,run_diff,base_state,cumulative_pitches
i64,datetime[μs],f64,f64,f64,str,i64,i64,i64,f64,i64,i64,i64,f64,i64,i64,i64,f64,f64,f64,f64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,f64,f64,f64,i64,i32,i32,i64
7,2024-10-20 00:00:00,78.8,4.1,5.27,"""Manaea, Sean""",571970,640455,4,14.0,1,0,2,-1.0,-1,3,1,-1.04,0.07,1.44,1.65,0,3,0,-1.0,-1.0,-3.876776,-114.790473,-1.213129,-8.519711,20.865706,-31.531093,2.99,1.5,-1.0,-1.0,-1.0,79.7,2141.0,6.8,775302,53.72,-1.0,0.689131,0.7,1.0,0.0,0.0,-1.0,31,5,2,0,294.0,-1.0,-1.0,0,-3,0,1
0,2024-10-20 00:00:00,83.9,4.16,5.13,"""Manaea, Sean""",571970,640455,-1,13.0,1,0,2,-1.0,-1,2,1,1.28,-0.01,-1.13,1.32,0,3,0,-1.0,-1.0,-14.910726,-121.300964,-2.306161,15.532788,23.944392,-32.030777,2.99,1.4,-1.0,-1.0,-1.0,84.7,1835.0,7.2,775302,53.31,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,31,4,2,0,106.0,-1.0,-1.0,1,-3,0,2
1,2024-10-20 00:00:00,91.9,4.03,4.97,"""Manaea, Sean""",571970,640455,-1,13.0,1,0,2,-1.0,-1,1,1,1.16,0.76,-0.93,2.17,0,3,0,-1.0,-1.0,-15.210428,-132.944842,-2.837582,16.955867,27.981449,-22.734943,2.99,1.53,-1.0,-1.0,-1.0,92.7,2131.0,7.0,775302,53.48,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,31,3,2,0,113.0,-1.0,-1.0,7,-3,0,3
7,2024-10-20 00:00:00,80.2,3.84,5.31,"""Manaea, Sean""",571970,640455,-1,13.0,1,0,2,-1.0,-1,0,1,-1.34,0.15,-0.86,1.28,0,3,0,-1.0,-1.0,-7.987162,-116.480556,-2.438514,-10.452965,23.959752,-30.532682,2.99,1.35,-1.0,-1.0,-1.0,81.3,2117.0,7.4,775302,53.14,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,31,2,2,0,284.0,-1.0,-1.0,2,-3,0,4
2,2024-10-20 00:00:00,91.3,4.16,5.09,"""Manaea, Sean""",669242,640455,7,11.0,0,0,0,-1.0,2,1,2,0.99,1.02,-0.05,3.52,0,3,0,73.3,43.22,-12.92281,-132.364096,-0.214144,14.481023,28.663536,-20.417379,3.27,1.5,406.0,104.1,31.0,92.4,2003.0,7.2,775302,53.31,0.846,1.646,2.0,1.0,0.0,3.0,6.0,30,5,0,0,122.0,73.42327,6.66579,0,-1,3,5
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
0,2024-03-16 00:00:00,84.0,1.71,6.4,"""Skubal, Tarik""",668804,669373,-1,14.0,0,0,1,-1.0,-1,1,1,1.52,0.6,1.09,2.08,0,1,0,-1.0,-1.0,-4.432459,-122.07474,-4.703139,16.032842,26.558863,-25.453648,3.3,1.67,-1.0,-1.0,-1.0,83.5,1505.0,6.3,747947,54.23,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6,3,0,1,130.0,-1.0,-1.0,0,0,3,68
0,2024-03-16 00:00:00,85.4,1.95,6.33,"""Skubal, Tarik""",668804,669373,-1,12.0,0,0,2,-1.0,-1,0,1,1.44,0.92,1.11,3.31,0,1,0,-1.0,-1.0,-4.854903,-124.173193,-2.600914,15.806897,27.350271,-22.359652,3.3,1.67,-1.0,-1.0,-1.0,84.9,1740.0,6.2,747947,54.26,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6,2,0,1,131.0,-1.0,-1.0,1,0,3,69
1,2024-03-16 00:00:00,96.2,1.78,6.3,"""Skubal, Tarik""",668804,669373,-1,2.0,0,0,1,-1.0,-1,0,0,1.37,1.08,-0.05,2.83,0,1,0,-1.0,-1.0,-7.948987,-139.714018,-5.985534,19.815637,34.920946,-16.847749,3.3,1.67,200.0,82.2,17.0,95.7,2161.0,6.4,747947,54.14,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6,1,0,1,145.0,-1.0,-1.0,0,0,3,70


In [70]:
# [ 'run_diff', 'base_state']


df = df.with_columns(
    (pl.col("fld_score") - pl.col("bat_score")).alias("run_diff").cast(pl.Int32),
)

df = df.drop(["fld_score", "bat_score"])
df.columns


['pitch_type',
 'game_date',
 'release_speed',
 'release_pos_x',
 'release_pos_z',
 'player_name',
 'batter',
 'pitcher',
 'events',
 'zone',
 'stand',
 'p_throws',
 'type',
 'hit_location',
 'bb_type',
 'balls',
 'strikes',
 'pfx_x',
 'pfx_z',
 'plate_x',
 'plate_z',
 'on_3b',
 'on_2b',
 'on_1b',
 'outs_when_up',
 'inning',
 'inning_topbot',
 'hc_x',
 'hc_y',
 'vx0',
 'vy0',
 'vz0',
 'ax',
 'ay',
 'az',
 'sz_top',
 'sz_bot',
 'hit_distance_sc',
 'launch_speed',
 'launch_angle',
 'effective_speed',
 'release_spin_rate',
 'release_extension',
 'game_pk',
 'release_pos_y',
 'estimated_ba_using_speedangle',
 'estimated_woba_using_speedangle',
 'woba_value',
 'woba_denom',
 'babip_value',
 'iso_value',
 'launch_speed_angle',
 'at_bat_number',
 'pitch_number',
 'if_fielding_alignment',
 'of_fielding_alignment',
 'spin_axis',
 'bat_speed',
 'swing_length',
 'next_pitch',
 'run_diff']

In [71]:
df = df.with_columns(
    pl.col("on_1b")
    .map_elements(lambda s: 0 if s == -1.0 else 1, return_dtype=pl.Int32)
    .alias("on_1b"),
    pl.col("on_2b")
    .map_elements(lambda s: 0 if s == -1.0 else 1, return_dtype=pl.Int32)
    .alias("on_2b"),
    pl.col("on_3b")
    .map_elements(lambda s: 0 if s == -1.0 else 1, return_dtype=pl.Int32)
    .alias("on_3b"),
)

df = df.with_columns(
    (pl.col("on_1b") * 3 + pl.col("on_2b") * 5 + pl.col("on_3b") * 7).alias(
        "base_state"
    )
)

df = df.drop(["on_1b", "on_2b", "on_3b"])
df

pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,zone,stand,p_throws,type,hit_location,bb_type,balls,strikes,pfx_x,pfx_z,plate_x,plate_z,outs_when_up,inning,inning_topbot,hc_x,hc_y,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,if_fielding_alignment,of_fielding_alignment,spin_axis,bat_speed,swing_length,next_pitch,run_diff,base_state
i64,datetime[μs],f64,f64,f64,str,i64,i64,i64,f64,i64,i64,i64,f64,i64,i64,i64,f64,f64,f64,f64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,f64,f64,f64,i64,i32,i32
0,2024-03-16 00:00:00,86.7,1.61,6.32,"""Skubal, Tarik""",457705,669373,0,12.0,0,0,0,8.0,0,0,2,1.35,0.6,0.93,2.69,0,1,0,109.79,117.8,-4.401027,-126.047957,-3.624164,15.246944,29.139518,-25.207848,3.12,1.57,174.0,85.7,12.0,86.3,1782.0,6.4,747947,54.12,0.66,-1.0,0.9,-1.0,1.0,0.0,4.0,5,3,0,0,130.0,-1.0,-1.0,1,0,0
1,2024-03-16 00:00:00,97.3,1.64,6.32,"""Skubal, Tarik""",457705,669373,-1,1.0,0,0,1,-1.0,-1,0,1,1.32,1.29,-0.64,2.65,0,1,0,-1.0,-1.0,-9.141577,-141.109845,-7.176998,19.856803,35.767834,-13.4769,3.12,1.57,-1.0,-1.0,-1.0,96.7,2123.0,6.4,747947,54.13,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,5,2,0,0,147.0,-1.0,-1.0,2,0,0
2,2024-03-16 00:00:00,94.7,1.62,6.33,"""Skubal, Tarik""",457705,669373,-1,6.0,0,0,1,-1.0,-1,0,0,0.77,1.32,0.43,2.3,0,1,0,-1.0,-1.0,-4.837722,-137.649177,-7.75787,10.937714,33.29673,-13.89737,3.12,1.57,-1.0,-1.0,-1.0,94.5,2038.0,6.4,747947,54.09,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,5,1,0,0,144.0,-1.0,-1.0,3,0,0
3,2024-03-16 00:00:00,79.1,1.87,6.32,"""Skubal, Tarik""",668804,669373,1,8.0,0,0,1,2.0,-1,2,2,-0.23,-0.51,0.0,1.71,0,1,0,-1.0,-1.0,-3.633845,-115.162656,-2.270099,-1.257192,23.623523,-36.409136,3.3,1.67,-1.0,-1.0,-1.0,78.8,2219.0,6.3,747947,54.23,-1.0,-1.0,0.0,-1.0,0.0,0.0,-1.0,6,6,0,1,330.0,-1.0,-1.0,2,0,3
2,2024-03-16 00:00:00,96.3,1.78,6.42,"""Skubal, Tarik""",668804,669373,-1,12.0,0,0,2,-1.0,-1,1,2,0.38,1.5,1.36,4.35,0,1,0,-1.0,-1.0,-1.97334,-140.109247,-3.246652,5.463838,36.528482,-11.994099,3.3,1.67,-1.0,-1.0,-1.0,95.5,2182.0,6.2,747947,54.33,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6,5,0,1,147.0,-1.0,-1.0,2,0,3
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
0,2024-10-20 00:00:00,82.2,4.03,5.2,"""Manaea, Sean""",669242,640455,-1,6.0,0,0,1,-1.0,-1,0,0,1.19,0.21,0.4,2.17,0,3,0,-1.0,-1.0,-10.620207,-119.311478,-0.712593,13.38854,22.522333,-30.227132,3.27,1.5,3.0,76.0,-39.0,83.0,1823.0,6.9,775302,53.59,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,30,1,0,0,108.0,79.3581,8.06376,7,-1,3
7,2024-10-20 00:00:00,78.8,4.1,5.27,"""Manaea, Sean""",571970,640455,4,14.0,1,0,2,-1.0,-1,3,1,-1.04,0.07,1.44,1.65,0,3,0,-1.0,-1.0,-3.876776,-114.790473,-1.213129,-8.519711,20.865706,-31.531093,2.99,1.5,-1.0,-1.0,-1.0,79.7,2141.0,6.8,775302,53.72,-1.0,0.689131,0.7,1.0,0.0,0.0,-1.0,31,5,2,0,294.0,-1.0,-1.0,0,-3,0
0,2024-10-20 00:00:00,83.9,4.16,5.13,"""Manaea, Sean""",571970,640455,-1,13.0,1,0,2,-1.0,-1,2,1,1.28,-0.01,-1.13,1.32,0,3,0,-1.0,-1.0,-14.910726,-121.300964,-2.306161,15.532788,23.944392,-32.030777,2.99,1.4,-1.0,-1.0,-1.0,84.7,1835.0,7.2,775302,53.31,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,31,4,2,0,106.0,-1.0,-1.0,1,-3,0


In [ ]:
df = df.with_columns(
    pl.col("pitch_type")
    .cum_count(reverse=False)
    .over(["player_name", "game_date", "game_pk"])
    .alias("pitches_thrown_curr_game")
)
df = df.sort(
    [
        "game_date",
        "game_pk",
        "at_bat_number",
        "pitch_number",
    ],
    descending=True,
)
df

pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,zone,stand,p_throws,type,hit_location,bb_type,balls,strikes,pfx_x,pfx_z,plate_x,plate_z,outs_when_up,inning,inning_topbot,hc_x,hc_y,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,if_fielding_alignment,of_fielding_alignment,spin_axis,bat_speed,swing_length,next_pitch,run_diff,base_state,cumulative_pitches,pitches_thrown_curr_game
i64,datetime[μs],f64,f64,f64,str,i64,i64,i64,f64,i64,i64,i64,f64,i64,i64,i64,f64,f64,f64,f64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,f64,f64,f64,i64,i32,i32,i64,u32
7,2024-10-20 00:00:00,78.8,4.1,5.27,"""Manaea, Sean""",571970,640455,4,14.0,1,0,2,-1.0,-1,3,1,-1.04,0.07,1.44,1.65,0,3,0,-1.0,-1.0,-3.876776,-114.790473,-1.213129,-8.519711,20.865706,-31.531093,2.99,1.5,-1.0,-1.0,-1.0,79.7,2141.0,6.8,775302,53.72,-1.0,0.689131,0.7,1.0,0.0,0.0,-1.0,31,5,2,0,294.0,-1.0,-1.0,0,-3,0,1,1
0,2024-10-20 00:00:00,83.9,4.16,5.13,"""Manaea, Sean""",571970,640455,-1,13.0,1,0,2,-1.0,-1,2,1,1.28,-0.01,-1.13,1.32,0,3,0,-1.0,-1.0,-14.910726,-121.300964,-2.306161,15.532788,23.944392,-32.030777,2.99,1.4,-1.0,-1.0,-1.0,84.7,1835.0,7.2,775302,53.31,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,31,4,2,0,106.0,-1.0,-1.0,1,-3,0,2,2
1,2024-10-20 00:00:00,91.9,4.03,4.97,"""Manaea, Sean""",571970,640455,-1,13.0,1,0,2,-1.0,-1,1,1,1.16,0.76,-0.93,2.17,0,3,0,-1.0,-1.0,-15.210428,-132.944842,-2.837582,16.955867,27.981449,-22.734943,2.99,1.53,-1.0,-1.0,-1.0,92.7,2131.0,7.0,775302,53.48,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,31,3,2,0,113.0,-1.0,-1.0,7,-3,0,3,3
7,2024-10-20 00:00:00,80.2,3.84,5.31,"""Manaea, Sean""",571970,640455,-1,13.0,1,0,2,-1.0,-1,0,1,-1.34,0.15,-0.86,1.28,0,3,0,-1.0,-1.0,-7.987162,-116.480556,-2.438514,-10.452965,23.959752,-30.532682,2.99,1.35,-1.0,-1.0,-1.0,81.3,2117.0,7.4,775302,53.14,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,31,2,2,0,284.0,-1.0,-1.0,2,-3,0,4,4
2,2024-10-20 00:00:00,91.3,4.16,5.09,"""Manaea, Sean""",669242,640455,7,11.0,0,0,0,-1.0,2,1,2,0.99,1.02,-0.05,3.52,0,3,0,73.3,43.22,-12.92281,-132.364096,-0.214144,14.481023,28.663536,-20.417379,3.27,1.5,406.0,104.1,31.0,92.4,2003.0,7.2,775302,53.31,0.846,1.646,2.0,1.0,0.0,3.0,6.0,30,5,0,0,122.0,73.42327,6.66579,0,-1,3,5,5
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
0,2024-03-16 00:00:00,84.0,1.71,6.4,"""Skubal, Tarik""",668804,669373,-1,14.0,0,0,1,-1.0,-1,1,1,1.52,0.6,1.09,2.08,0,1,0,-1.0,-1.0,-4.432459,-122.07474,-4.703139,16.032842,26.558863,-25.453648,3.3,1.67,-1.0,-1.0,-1.0,83.5,1505.0,6.3,747947,54.23,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6,3,0,1,130.0,-1.0,-1.0,0,0,3,68,68
0,2024-03-16 00:00:00,85.4,1.95,6.33,"""Skubal, Tarik""",668804,669373,-1,12.0,0,0,2,-1.0,-1,0,1,1.44,0.92,1.11,3.31,0,1,0,-1.0,-1.0,-4.854903,-124.173193,-2.600914,15.806897,27.350271,-22.359652,3.3,1.67,-1.0,-1.0,-1.0,84.9,1740.0,6.2,747947,54.26,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6,2,0,1,131.0,-1.0,-1.0,1,0,3,69,69
1,2024-03-16 00:00:00,96.2,1.78,6.3,"""Skubal, Tarik""",668804,669373,-1,2.0,0,0,1,-1.0,-1,0,0,1.37,1.08,-0.05,2.83,0,1,0,-1.0,-1.0,-7.948987,-139.714018,-5.985534,19.815637,34.920946,-16.847749,3.3,1.67,200.0,82.2,17.0,95.7,2161.0,6.4,747947,54.14,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6,1,0,1,145.0,-1.0,-1.0,0,0,3,70,70
